# BigEarthNet model
In this notebook we implement a deep learning model for the classification of pixels in satellite images. We use the same model implemented for the EPFL Building footprints. The dataset for training the model is [BigEarthNet](https://bigearth.net/#downloads). The original TIFF images for the Sentinel-2 bands and the relative masks have been transformed into PNG files.

In [2]:
import os
from pathlib import Path
from zipfile import ZipFile
import matplotlib.pyplot as plt
from skimage import io
from skimage import exposure
from skimage.io import imsave, imread
import numpy as np
from sklearn.metrics import jaccard_score
#import tensorflow as tf
#import keras
#from keras import layers
#from tensorflow.keras.utils import array_to_img, load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')
print("NumPy version: %s"%np.__version__)
#print('Tensorflow version: {:s}'.format(tf.__version__))
#print('Keras version. {}'.format(keras.__version__))

NumPy version: 1.26.4


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 8.5 gigabytes of available RAM



## The dataset
The RGB PNG files and the corresponding masks are available as zip files. They can be downloaded from an S3 bucket on AWS

In [1]:
DATADIR = '.'
presigned_url = 'https://selmilab-bucket.s3.eu-south-1.amazonaws.com/'

In [ ]:
!wget -nv presigned_url -P DATADIR presigned_url

In [ ]:
with ZipFile(f'{DATADIR}/epfl_buildings_train.zip', 'r') as zipObj:
   zipObj.extractall(path=f'{DATADIR}/')

In [ ]:
with ZipFile(f'{DATADIR}/epfl_buildings_test.zip', 'r') as zipObj:
   zipObj.extractall(path=f'{DATADIR}/')